# 2.2 Data Description Report

Here, we'll provide a detailed description of the collected data:

* A detailed description of each dataset.
* Number of instances and attributes.
* Identification of target features.

The initial transformations to the data that will take place are:

* Setting datatypes for dataframe columns.
* Addition of various time fields to aid data exploration.
* Isolate a single point location for data exploration (the intersection of Northumberland Street and Saville Row with cameras looking both east and west).

In [1]:
# importing required libraries
import pandas as pd
import numpy as np
import os
import re
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, Normalize

# path to data files
RAW_DATA_PATH = "../data/tom_komar_csv_22_23/"

# dictionary to store dataframes
dfs = {}

# iterating over all files in the directory
for file in os.listdir(RAW_DATA_PATH):
    # reading file into a dataframe
    df = pd.read_csv(os.path.join(RAW_DATA_PATH, file))
    # extracting date from the filename
    date = "-".join(re.split(r"[-.]", file)[-3:-1])
    # storing dataframe in the dictionary with date as the key
    dfs[date] = df

# concatenating all dataframes into a single dataframe
concat_df = pd.concat([df.assign(key=key) for key, df in dfs.items()])

The raw csv contains extra locations that are not of interest so these will need to be filtered out. The raw data tells us we are dealing with pedestrian flow in the form $n/15m$ where $n$ is the number of pedestrians and $m$ is minutes. We can also see that each row specifies a directional vector. 

In [2]:
# display raw data and data types
print("Raw data:")
display(concat_df.sample(5, random_state=42))
print()
print("Raw column data types:")
df = pd.DataFrame(concat_df.dtypes)
df.columns = ["dtype"]
display(df)

Raw data:


,dt,value,veh_class,dir,location,category,key
10747,2023-03-11 19:15:00,5,person,from_north_to_south,NclQueenVictoriaRdPercyBuilding,flow,2023-03
14403,2022-08-18 02:45:00,4,person,southwest_to_northwest,NclNorthumberlandStSavilleRowWest,flow,2022-08
11185,2022-06-11 20:15:00,9,person,northwest_to_southwest,NclNorthumberlandStSavilleRowWest,flow,2022-06
1724,2022-03-02 23:15:00,9,person,northeast_to_southeast,NclNorthumberlandStSavilleRowEast,flow,2022-03
26912,2022-12-31 00:45:00,6,person,north_to_south,NclNorthumberlandStThePearl,flow,2022-12



Raw column data types:


,dtype
dt,object
value,int64
veh_class,object
dir,object
location,object
category,object
key,object


In [3]:
# filtering data for specific locations
concat_df = concat_df[
    concat_df["location"].isin(
        ["NclNorthumberlandStSavilleRowEast", "NclNorthumberlandStSavilleRowWest"]
    )
]

# splitting 'dt' column into 'date' and 'time' columns
concat_df[["date", "time"]] = concat_df["dt"].str.split(" ", expand=True)

# converting 'dt', 'date', 'time' columns to datetime format
concat_df["dt"] = pd.to_datetime(concat_df["dt"])
concat_df["date"] = pd.to_datetime(concat_df["date"])
concat_df["time"] = pd.to_datetime(concat_df["time"], format="%H:%M:%S")

We will create a few new columns that are string formatted derivatives of the datetime column that will help with visualising the data. We will also create some integer columns for day in week, month, quarter, and hours and minutes since midnight. This is to assist in any later analysis. 

In [4]:
# extracting various time units from 'dt' column
concat_df["hour-minutes"] = concat_df["dt"].dt.strftime("%H:%M")
concat_df["year-month-day-hour"] = concat_df["dt"].dt.strftime("%Y-%m-%d %H")
concat_df["year-month-hour"] = concat_df["dt"].dt.strftime("%Y-%m %H")
concat_df["month-hour"] = concat_df["dt"].dt.strftime("%m %H")
concat_df["year-month"] = concat_df["dt"].dt.strftime("%Y-%m")
concat_df["year-week"] = concat_df["date"].dt.strftime("%Y-%U")
concat_df["hours_since_midnight"] = concat_df["dt"].dt.hour
concat_df["minutes_since_midnight"] = concat_df["time"].apply(
    lambda t: t.hour * 60 + t.minute
)
concat_df["year-quarter"] = concat_df["dt"].apply(
    lambda x: f"{x.year}-{(x.month - 1) // 3 + 1}"
)
concat_df["month"] = concat_df["date"].dt.month
concat_df["quarter"] = concat_df["date"].dt.quarter
concat_df["day_in_week"] = concat_df["date"].dt.dayofweek
concat_df["day_of_week"] = concat_df["date"].dt.day_name()

The next step is to filter the data on the location field as we are only interested in the NclNorthumberlandStSavilleRowEast and NclNorthumberlandStSavilleRowWest sensors initially. We will save these as seperate dataframes `east_df` and `west_df` and pickle them to preserve column data types when we need to read them in other notebooks.

In [5]:
# dividing data into east and west dataframes
east_df = concat_df[concat_df["location"] == "NclNorthumberlandStSavilleRowEast"]
west_df = concat_df[concat_df["location"] == "NclNorthumberlandStSavilleRowWest"]

DATA_PATH = "../data/saville_row_east_west/"

east_df.to_csv(os.path.join(DATA_PATH, "east_df.csv"))
west_df.to_csv(os.path.join(DATA_PATH, "west_df.csv"))

east_df.to_pickle(os.path.join(DATA_PATH, "east_df.pkl"))
west_df.to_pickle(os.path.join(DATA_PATH, "west_df.pkl"))

In [6]:
print("New columns and data types:")
df = pd.DataFrame(east_df.dtypes)
df.columns = ["dtype"]
display(df)

New columns and data types:


,dtype
dt,datetime64[ns]
value,int64
veh_class,object
dir,object
location,object
category,object
key,object
date,datetime64[ns]
time,datetime64[ns]
hour-minutes,object


Now lets have a look at some of the attributes and features that are of interest. We can see that the data has significant gaps - around 15% of the data over this period is missing if we calculate the number of 15 minute periods that exist between the start and end dates. We can also see that each sensor direction contains two directional vectors. These describe whether a persons direction of travel is left or right across the CCTV footage. 

In [7]:
print(f"East | Length: {len(east_df)}")
print(f"West | Length: {len(west_df)}")
print()
print(f"East | Start date: {east_df['dt'].min()} | End date: {east_df['dt'].max()}")
print(f"West | Start date: {west_df['dt'].min()} | End date: {west_df['dt'].max()}")
print()
diff = east_df["dt"].max() - east_df["dt"].min()
intervals = int(diff.total_seconds() // (15 * 60))

print(f"East | Length if complete: {intervals*2}")
print(f"West | Length if complete: {intervals*2}")
print()

print(f"East | Completeness: {len(east_df)/(2*intervals)*100:.1f}%")
print(f"West | Completeness: {len(west_df)/(2*intervals)*100:.1f}%")
print()
print(
    f"East | Null count for flow: {east_df['value'].isna().sum()} | Null count for datetime: {east_df['dt'].isna().sum()}"
)
print(
    f"West | Null count for flow: {west_df['value'].isna().sum()} | Null count for datetime: {west_df['dt'].isna().sum()}"
)

East | Length: 79522
West | Length: 79787

East | Start date: 2021-12-02 14:00:00 | End date: 2023-03-31 23:45:00
West | Start date: 2021-12-02 14:00:00 | End date: 2023-03-31 23:45:00

East | Length if complete: 93006
West | Length if complete: 93006

East | Completeness: 85.5%
West | Completeness: 85.8%

East | Null count for flow: 0 | Null count for datetime: 0
West | Null count for flow: 0 | Null count for datetime: 0

East | Directional vectors: ['northeast_to_southeast' 'southeast_to_northeast']
West | Directional vectors: ['northwest_to_southwest' 'southwest_to_northwest']


In [ ]:
print()
print(f"East | Directional vectors: {east_df['dir'].unique()}")
print(f"West | Directional vectors: {west_df['dir'].unique()}")